<a href="https://colab.research.google.com/github/Zufiqqar/MachineLearning/blob/main/10_Decision_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Available at https://www.comp.nus.edu.sg/~winkler/IT3011/decision-trees.html

<h1>NUS IT3011 Machine Learning and Applications</h1>

<h2>Decision Trees</h2>


In the notebook, we implement a decision tree from scratch and see how well it works.


---
# 1 Implement Decision Tree

We are going to implement a decision tree that can classify whether a salary of a person is above 50k USD per year or not (so it is a 2-class classification).

We will use the adult dataset (a.k.a. census income dataset) from UCI: https://archive.ics.uci.edu/ml/datasets/adult .

In [ ]:
import csv
import math
import random
import numpy as np
import pandas as pd

### .a Generate Dataset

We are going to explore the dataset. We are going to make use of 14 _attributes_ (also known as _features_ or _input dimensions_, but for this decision trees to be consistent with the algorithm, we'll use "attributes"). The attributes at index \[0, 2, 4, 10, 11, 12] are continuous. We are going to discretize the continuous features, where values **larger** than the mean is assigned **1** and values **smaller** than the mean is assigned **0**.

In [ ]:
def generate_data(data_type):
  """
   Args:
     data_type(String): "train" or "test"
      
   Returns:
     data (np array) : contains the dataset loaded from csv file
  """
  attributes = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-info_gain', 'capital-loss', 'hours-per-week', 'native-country', 'salary']
     
  if(data_type == 'train'):
    data = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    names=attributes,
    sep=r'\s*,\s*',
    engine='python',
    na_values="?")
  else:
    data = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
    names=attributes,
    sep=r'\s*,\s*',
    skiprows=[0],
    engine='python',
    na_values="?")
    
  data = data.values
  data = np.array(data)
  
  continuous_index = [0, 2, 4, 10, 11, 12]

  for j in continuous_index:
    mean_value=np.mean(data[:, j].astype(float))
    for i in range(data.shape[0]):
      if(data[i, j] > mean_value):
        data[i, j] = '1'
      else:
        data[i, j] = '0'
       
  return data


In [ ]:
sample_data = generate_data("train")[:10]
pd.DataFrame(sample_data)

### .b Choose Feature Based on Majority

Next we are going to build a list of helper functions to help build the decision tree. Your task is to implement these functions.

**Majority Class Function**: When you reach a terminal state in a decision tree, we return the value that represents the majority of the (sub)-dataset's instances of the class as the answer.

**Yout Turn (Question 3):** Complete the code below to find the majority class from the dataset.

In [ ]:
def major_class(data):
    """ 
    Args:
      data(m, K): The current sub-dataset of the sub-tree
          m: num_rows
          K: num_features + 1 (last column is the label)
    Returns:
      major: Type:String. The major class of this sub-dataset(e.g ">50K" or "<=50K")
    """
    major = ""
    
    # freq will store the number of occurences of the target labels
    freq = {}
    for entry in data:
        if (entry[-1] in freq):
            freq[entry[-1]] += 1.0
        else:
            freq[entry[-1]]  = 1.0
            
    major = ""
    ###########################
    #
    # Your Turn (Q3): Write your code here
    # Hint: Loop through each row of data, get the last column, then find which value occurs the most frequently.
    #
    ###########################

    return major
  

# Testing: This should return '<=50K'
major_class(sample_data)

### .c Calculate Entropy

**Entropy Function**: Calculate the entropy of this current (sub-)dataset:

Recall: $H(X) = - \sum_{i=0}^C p_i\log(p_i)$

**Your Turn (Question 4):** Complete the code below to calculate entropy of the dataset.

In [ ]:
def entropy(data):
    """ 
    Args:
      data(m, K): The current sub-dataset of the sub-tree
          m: num_rows
          K: num_features + 1 (last column is the label)
    Returns:
      dataEntropy: The entropy of this current sub-dataset
    """
    dataEntropy = 0.0
    # freq will store the number of occurences of the target labels
    freq = {}
    for entry in data:
        if (entry[-1] in freq):
            freq[entry[-1]] += 1.0
        else:
            freq[entry[-1]]  = 1.0
    ###########################
    #
    # Your Turn (Q4): Write your code here
    # Hint: Loop through each row of data, get the last column, find number of occurences of each value, then use the above equation.
    #
    ###########################
    
    return dataEntropy

# Testing: This should return 0.8812908992306927 (in log 2)
entropy(sample_data)

### .d Calculate Information Gain

**Information Gain**: The information gained by splitting the current (sub)-dataset using the attribute.

Recall:  Information Gain is a metric that measures the expected reduction in the impurity of the collection $S$, caused by splitting the data according to any given attribute. A chosen feature $x_i$ divides the example set S into subsets
$S_1 , S_2 , ... , S_{C_i}$ according to the $C_i$ distinct values for $x_i$ .
The entropy then reduces to the entropy of the subsets $S_1 , S_2 , ... , S_{C_i}$:

<div align="center">
$\text{remainder}(S, x_i) = \sum_{j=1}^{C_i} \frac{|S_j|}{|S|} H(S_j)$
</div>

Information Gain (IG; “reduction in entropy”) from knowing the value of $x_i$. We choose the attribute with the largest IG:
<div align="center">
$IG(S, x_i) = H(S) - \text{remainder}(S, x_i) $  
</div>

**Your Turn (Question 5):** Complete the code below to calculate information gain of a given attribute.

In [ ]:
def info_gain(data, attribute, attributes):
    """ 
    Args:
      data(m, K): The current sub-dataset of the sub-tree
          m: num_rows
          K: num_features + 1 (last column is the label)
      attribute: The attribute used to split data
      attributes: The list of current remaining attributes
    Returns:
      info_gain : information gain of the given dataset
    """
    
    freq = {}
    subsetEntropy = 0.0
    
    # Get the column index of this attribute
    i = attributes.index(attribute)
    
    for entry in data:
        if (entry[i] in freq):
            freq[entry[i]] += 1.0
        else:
            freq[entry[i]]  = 1.0

    ###########################
    #
    # Your Turn (Q5): Write your code here
    # Hint: Split the data based on the value at index i. Find the subsetEntropy of
    # each sub-dataset, then use the formula of Information Gain to calculate subsetEntropy.
    #
    ###########################
    return (entropy(data) - subsetEntropy)

# Testing: This should return 0.0058021490143458365
attributes = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-info_gain', 'capital-loss', 'hours-per-week', 'native-country', 'salary']
info_gain(sample_data, 'age', attributes)

### .e Find the Best Attribute

Now we will write a function to choose the **best** attribute for a given data, here best indicates having **largest** information gain among all attributes.

**Your Turn (Question 6):** Complete the code below to get the best attribute based on information gain.

In [ ]:
def attr_choose(data, attributes):
    """ 
    Args:
      data(m, K): The current sub-dataset of the sub-tree
          m: num_rows
          K: num_features + 1 (last column is the label)
      attributes: The list of current remaining attributes
    Returns:
      best: The best attribute to split based on info gain.
    """
    best = attributes[0]
    
    maxGain = 0
    for attr in attributes[:-1]:
      ###########################
      #
      # Your Turn (Q6): Write your code here
      # Hint: For each attribute in attributes, use Info_gain function above
      # to know which attribute is the best option
      #
      ###########################
    return best
  
# Testing: This should return 'hours-per-week'
attributes = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-info_gain', 'capital-loss', 'hours-per-week', 'native-country', 'salary']
attr_choose(sample_data, attributes)

### .f Helper Functions

We will define two helper functions here. First, `get_unique_values` returns the unique values of a given attribute. The second function, `get_data` returns the rows containing a specific value of a chosen attribute.

In [ ]:
# These two functions are helper functions
# This function will get unique values for that particular attribute from the given data
def get_unique_values(data, attributes, attribute):
    """
    Args:
      data (m,K) : Current subset of data
      attributes : The list of current remaining attributes
      attribute : Our point of interest
    
    Returns:
      values : List of unique values for our point of interest
    
    """
    index = attributes.index(attribute)
    values = []
    # 
    for entry in data:
        if entry[index] not in values:
            values.append(entry[index])

    return values

# This function will get all the rows of the data where the chosen "best" attribute has a value "val"
def get_data(data, attributes, best, val):
    """
    Args:
      data (m,K) : Current subset of data
      attributes : The list of current remaining attributes
      best : The attribute of which data we will extract
      val : We are interested only on this value of the `best` attribute
    Returns:
      new_data : Data subset containing only those rows where `best` attribute = val
    
    """
    new_data = [[]]
    index = attributes.index(best)

    for entry in data:
        if (entry[index] == val):
            newEntry = []
            for i in range(0,len(entry)):
                if(i != index):
                    newEntry.append(entry[i])
            new_data.append(newEntry)

    new_data.remove([])    
    return new_data


### .g Build Decision Tree

In the below code, your task is to implement the build a tree recursively. Starting at the root, pick the best attribute to split on, and call the `build_tree` function on each of the sub-trees.  Check the inlined code comments for clarification.

**Your Turn (Question 7):** Complete the code below to build the tree.

In [ ]:
def build_tree(data, attributes):
    """ 
      Args:
        data(m, K): The current sub-dataset of the sub-tree
            m: num_rows
            K: num_features + 1 (last column is the label)
        attributes: The list of current remaining attributes
      Returns:
        tree: The constructed tree as object. For example if the root is gender,
              then a tree of depth 2 is like 
              {'gender': {'male': sub_tree1, 'female': sub_tree2}}
    """
    data = data[:]
    
    # vals is the labels of the records in data.
    vals = [record[-1] for record in data]
    
    
    ##################################################################
    ## Your Turn (Q7): Finish the implementation of the below code ##
    if (len(data) == 0) or (len(attributes) - 1) <= 0: # What should we return if we run out of features to split?
        return #replace with your answer
    elif vals.count(vals[0]) == len(vals): # What should we return if this sub-dataset is pure?
        return #replace with your answer
    else:
        best = attr_choose(data, attributes)
        tree = {best:{}}
    
        for val in get_unique_values(data, attributes, best):
            ### Calculate the subtree here
            #
            # Write your code here
            #
            tree[best][val] = subtree
    
    return tree

The below code block containing `run_decision_tree` does exactly as its name, and you don't have to understand its code for the purpose of this exercise. Just run it and check the accuracy.

In [ ]:
# Class Node which will be used while classify a test-instance using the tree which was built earlier
class Node():
    value = ""
    children = []

    def __init__(self, val, dictionary):
        self.value = val
        if (isinstance(dictionary, dict)):
            self.children = list(dictionary.keys())

def run_decision_tree():
    
#     data = generate_data()
    attributes = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-info_gain', 'capital-loss', 'hours-per-week', 'native-country', 'salary']
    acc = []
    training_set = generate_data("train")
    test_set = generate_data("test")
    tree = build_tree(training_set, attributes)
    results = []

    for entry in test_set:
        tempDict = tree.copy()
        result = ""
        while(isinstance(tempDict, dict)):
            root = Node(list(tempDict.keys())[0], tempDict[list(tempDict.keys())[0]])
            tempDict = tempDict[list(tempDict.keys())[0]]
            index = attributes.index(root.value)
            value = entry[index]
            if(value in list(tempDict.keys())):
                child = Node(value, tempDict[value])
                result = tempDict[value]
                tempDict = tempDict[value]
            else:
                result = "Null"
                break
        if result != "Null":
            results.append(result == entry[-1][:-1])
        
    accuracy = float(results.count(True))/float(len(results))
    print(results)
    print("FINAL ACCURACY")
    print(accuracy)

In [ ]:
# run decision tree to get the accuracy
run_decision_tree()

# 2 Visualize Decision Tree Boundaries


Let's apply Decision Trees, Random Forests and the AdaBoost classifiers with
Decision Stumps to classify on the Iris dataset. We'll visualize the decision boundary as well.

For the sake of visualization, we will choose subsets of two features from  the Iris dataset. 

### .a Code to be run

The code block below can be safely folded away and run.  While you're welcome to study the code, it is not necessary to understand it entirely. All you need to do is to modify the below variable `trial_pairs` to play around with the code.

In [ ]:
##################################################################
### Your Turn: modify and try different feature combinations  ####
trial_pairs = ([0, 1], [0, 2], [2, 3])

##################################################################

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from sklearn import clone
from sklearn.datasets import load_iris
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier,
                              AdaBoostClassifier)
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# You may have to change these parameters later, to answer Q5-6
# Parameters
n_classes = 3
n_estimators = 30  # no of estimaotr for random forest and adaboost
cmap = plt.cm.RdYlBu
plot_step = 0.02  # fine step width for decision surface contours
plot_step_coarser = 0.5  # step widths for coarse classifier guesses
RANDOM_SEED = 13  # fix the seed on each iteration

In [ ]:
# Load data
iris = load_iris()

In [ ]:
models = [DecisionTreeClassifier(max_depth=None),
          RandomForestClassifier(n_estimators=n_estimators),
          AdaBoostClassifier(DecisionTreeClassifier(max_depth=5),
                             n_estimators=n_estimators)]

plot_idx = 1
for pair in trial_pairs:
    X = iris.data[:, pair]
    y = iris.target
    
    plt.subplot(1, 3, plot_idx)
    plt.scatter(X[:, 0], X[:, 1], c=y,
                    cmap=ListedColormap(['r', 'y', 'b']),
                    edgecolor='k', s=20)
    plt.title(pair)
    plot_idx += 1  # move on to the next plot in sequence
plt.show()

In [ ]:
plot_idx = 1


#######################################
for pair in trial_pairs:
    for model in models:
      
        # We only take the two corresponding features
        X = iris.data[:, pair]
        y = iris.target

        # Shuffle
        idx = np.arange(X.shape[0])
        np.random.seed(RANDOM_SEED)
        np.random.shuffle(idx)
        X = X[idx]
        y = y[idx]

        # Standardize
        mean = X.mean(axis=0)
        std = X.std(axis=0)
        X = (X - mean) / std
        
        # Train
        clf = clone(model)
        clf = model.fit(X, y)
        scores = clf.score(X, y)
        
        # Create a title for each column and the console by using str() and
        # slicing away useless parts of the string
        model_title = str(type(model)).split(".")[-1][:-2][:-len("Classifier")]
        model_details = model_title
        if hasattr(model, "estimators_"):
            model_details += " with {} estimators".format(
                len(model.estimators_))
        print(model_details + " with features", pair,
              "has a score of", scores)

        plt.subplot(3, 3, plot_idx)
        if plot_idx <= len(models):
            # Add a title at the top of each column
            plt.title(model_title)

        # Now plot the decision boundary using a fine mesh as input to a
        # filled contour plot
        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                             np.arange(y_min, y_max, plot_step))

        # Plot either a single DecisionTreeClassifier or alpha blend the
        # decision surfaces of the ensemble of classifiers
        if isinstance(model, DecisionTreeClassifier):
            Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
            Z = Z.reshape(xx.shape)
            cs = plt.contourf(xx, yy, Z, cmap=cmap)
        else:
            # Choose alpha blend level with respect to the number
            # of estimators
            # that are in use (noting that AdaBoost can use fewer estimators
            # than its maximum if it achieves a good enough fit early on)
            estimator_alpha = 1.0 / len(model.estimators_)
            for tree in model.estimators_:
                Z = tree.predict(np.c_[xx.ravel(), yy.ravel()])
                Z = Z.reshape(xx.shape)
                cs = plt.contourf(xx, yy, Z, alpha=estimator_alpha, cmap=cmap)

        # Build a coarser grid to plot a set of ensemble classifications
        # to show how these are different to what we see in the decision
        # surfaces. These points are regularly space and do not have a
        # black outline
        xx_coarser, yy_coarser = np.meshgrid(
            np.arange(x_min, x_max, plot_step_coarser),
            np.arange(y_min, y_max, plot_step_coarser))
        Z_points_coarser = model.predict(np.c_[xx_coarser.ravel(),
                                         yy_coarser.ravel()]
                                         ).reshape(xx_coarser.shape)
        cs_points = plt.scatter(xx_coarser, yy_coarser, s=15,
                                c=Z_points_coarser, cmap=cmap,
                                edgecolors="none")

        # Plot the training points, these are clustered together and have a
        # black outline
        plt.scatter(X[:, 0], X[:, 1], c=y,
                    cmap=ListedColormap(['r', 'y', 'b']),
                    edgecolor='k', s=20)
        plot_idx += 1  # move on to the next plot in sequence

plt.suptitle("Classifiers on feature subsets of the Iris dataset")
plt.axis("tight")
plt.show()

### .b Questions for your understanding

**Your Turn (Question 5):** From the visualization above, is Random Forest doing better than Decision Tree? Will the result still be the same, or improve if we have less estimators (say $10$) for Random Forest?


**Your Turn (Question 6):** You might have found, we use `RANDOM_SEED = 13` to get a deterministic result for everyone. What if we change the seed to a different value, say `RANDOM_SEED = 119`. Will the performance of three classifiers, decision tree, random forest and adaboost will still be same? If not, why?


**Your Turn (Question 7):**  There are other feature pairs in the Iris dataset. Experiment with these additional feature pairs to see which group of features perfom best among all pairs.


---
# Credits
Authored by Le Trung Hieu, Mohammad Neamul Kabir, Aadyaa Maddi and [Min-Yen Kan](http://www.comp.nus.edu.sg/~kanmy) (2019), affiliated with [WING](http://wing.comp.nus.edu.sg), [NUS School of Computing](http://www.comp.nus.edu.sg) and [ALSET](http://www.nus.edu.sg/alset).
Licensed as: [Creative Commons Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0/ ) (CC BY 4.0).
Please retain and add to this credits cell if using this material as a whole or in part.
Other Credits (inclusive of photos): 